# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 10:49AM,242348,20,6495133,"ACI Healthcare USA, Inc.",Released
1,Jul 21 2022 10:49AM,242348,20,6605147,"ACI Healthcare USA, Inc.",Released
2,Jul 21 2022 10:49AM,242348,20,8138310,"ACI Healthcare USA, Inc.",Released
3,Jul 21 2022 10:49AM,242348,20,8199002,"ACI Healthcare USA, Inc.",Released
4,Jul 21 2022 10:49AM,242348,20,8199004,"ACI Healthcare USA, Inc.",Released
5,Jul 21 2022 10:49AM,242348,20,8199011,"ACI Healthcare USA, Inc.",Released
6,Jul 21 2022 10:49AM,242348,20,8199014,"ACI Healthcare USA, Inc.",Released
7,Jul 21 2022 10:49AM,242348,20,8199018,"ACI Healthcare USA, Inc.",Released
8,Jul 21 2022 10:49AM,242348,20,8199020,"ACI Healthcare USA, Inc.",Released
9,Jul 21 2022 10:49AM,242348,20,8199022,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,242413,Released,3
29,242415,Released,9
30,242417,Released,1
31,242418,Released,1
32,242419,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242413,NaN,3.0
242415,NaN,9.0
242417,NaN,1.0
242418,NaN,1.0
242419,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242319,0.0,1.0
242325,0.0,1.0
242328,0.0,1.0
242336,0.0,1.0
242340,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242319,0,1
242325,0,1
242328,0,1
242336,0,1
242340,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242319,0,1
1,242325,0,1
2,242328,0,1
3,242336,0,1
4,242340,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242319,,1
1,242325,,1
2,242328,,1
3,242336,,1
4,242340,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc."
78,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.
87,Jul 21 2022 10:34AM,242419,19,ACG North America LLC
88,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions
89,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc."
90,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc."
93,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation
105,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service
109,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.
110,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",,78
1,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,,9
2,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,,1
3,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,,1
4,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",,1
5,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",,3
6,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,,12
7,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,,4
8,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.,,1
9,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78,
1,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,9,
2,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,1,
3,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,1,
4,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",1,
5,Jul 21 2022 10:13AM,242413,20,"Exact-Rx, Inc.",3,
6,Jul 21 2022 10:07AM,242405,10,ISDIN Corporation,12,
7,Jul 21 2022 10:03AM,242411,16,Sartorius Lab Products and Service,4,
8,Jul 21 2022 9:56AM,242410,16,Sartorius Stedim Filters Inc.,1,
9,Jul 21 2022 9:49AM,242408,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78,
1,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,9,
2,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,1,
3,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,1,
4,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78.0,NaN
1,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,9.0,NaN
2,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,1.0,NaN
3,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,1.0,NaN
4,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 10:49AM,242348,20,"ACI Healthcare USA, Inc.",78.0,0.0
1,Jul 21 2022 10:41AM,242415,10,Eywa Pharma Inc.,9.0,0.0
2,Jul 21 2022 10:34AM,242419,19,ACG North America LLC,1.0,0.0
3,Jul 21 2022 10:32AM,242418,16,Sartorius Bioprocess Solutions,1.0,0.0
4,Jul 21 2022 10:31AM,242417,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2423914,137.0,19.0
12,242336,1.0,0.0
16,1454450,9.0,0.0
19,969500,16.0,1.0
20,727162,81.0,6.0
21,1696490,7.0,0.0
22,242417,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2423914,137.0,19.0
1,12,242336,1.0,0.0
2,16,1454450,9.0,0.0
3,19,969500,16.0,1.0
4,20,727162,81.0,6.0
5,21,1696490,7.0,0.0
6,22,242417,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,137.0,19.0
1,12,1.0,0.0
2,16,9.0,0.0
3,19,16.0,1.0
4,20,81.0,6.0
5,21,7.0,0.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,1.0
2,16,Released,9.0
3,19,Released,16.0
4,20,Released,81.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Executing,19.0,0.0,0.0,1.0,6.0,0.0,0.0
Released,137.0,1.0,9.0,16.0,81.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Executing,19.0,0.0,0.0,1.0,6.0,0.0,0.0
1,Released,137.0,1.0,9.0,16.0,81.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Executing,19.0,0.0,0.0,1.0,6.0,0.0,0.0
1,Released,137.0,1.0,9.0,16.0,81.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()